# Preprocess

## Imports

In [1]:
import pandas as pd
from IPython.display import clear_output

In [2]:
from Dude import PATH, tp, stp, features

In [3]:
%load_ext autoreload
%autoreload 2
clear_output()

# Workspace

In [4]:
table = pd.merge( 
                pd.read_csv(f"{PATH}{tp}.txt", low_memory=False)  ,
                pd.read_csv(f"{PATH}{stp}.txt", low_memory=False) ,
                on = "trip_id"
                ).loc[:,features]

In [5]:
rids = table.route_id.unique()
sids = table.stop_id.unique()

In [6]:
print(
    f"""
    Number of identical routes: {rids.size}
    Number of identical stops: {sids.size}
    """
)


    Number of identical routes: 353
    Number of identical stops: 5388
    


In [16]:
routes_with_stops = table.set_index(["route_id","stop_id"])

In [18]:
print(f"The stops in route id : {rids[0]}")
routes_with_stops.loc[rids[0],slice(None)]

The stops in route id : 0078


,stop_sequence,direction_id
stop_id,,
008569,0,0
F03291,1,0
F03403,2,0
F03402,3,0
F03400,4,0
...,...,...
F03201,24,0
F03209,25,0
F03154,26,0
